In [2]:
import pandas as pd
import numpy as np
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from time import time

datasets = pd.read_csv("583_fri_c1_1000_50.tsv.gz", sep='\t')
datasets.head()


,oz1,oz2,oz3,oz4,oz5,oz6,oz7,oz8,oz9,oz10,...,oz42,oz43,oz44,oz45,oz46,oz47,oz48,oz49,oz50,target
0,0.983944,1.249931,0.014834,-0.273650,0.535293,-0.045385,0.733688,-0.130951,1.148869,0.124094,...,0.303152,1.656565,-1.646551,1.262836,0.930126,0.940371,-0.071487,-0.074410,0.961796,-1.632374
1,-0.156061,0.591306,-1.397949,0.987841,-1.231686,-0.887390,-1.633301,0.735855,-1.385308,0.255703,...,-1.432762,1.241678,-0.941083,-1.575093,1.693784,-1.159743,-1.362521,-0.700582,0.622595,0.269932
2,-0.624560,-0.233004,-1.132794,-0.139077,0.663901,0.659143,-0.134707,0.366280,0.482799,-0.228204,...,1.458085,-1.719528,-1.177299,-0.210058,0.519463,-1.198767,0.897829,-0.077448,-1.426452,1.298223
3,-0.231424,0.176015,-1.088738,0.850180,-0.242132,0.420693,0.995372,-0.928780,0.418453,-0.577191,...,-0.423482,1.231547,1.018242,0.425738,-1.275904,1.498841,1.043248,-0.553736,-0.255119,1.247661
4,-0.662773,0.185382,-1.310413,1.238650,1.681172,-0.041958,-0.094359,-1.551061,1.283978,-0.008615,...,0.891956,1.720278,-0.685381,0.929598,-1.597108,-0.019599,-0.127696,-1.513206,-0.082542,1.966738


In [3]:
X = datasets.drop("target", axis=1)
y = datasets['target']


,oz1,oz2,oz3,oz4,oz5,oz6,oz7,oz8,oz9,oz10,...,oz41,oz42,oz43,oz44,oz45,oz46,oz47,oz48,oz49,oz50
0,0.983944,1.249931,0.014834,-0.273650,0.535293,-0.045385,0.733688,-0.130951,1.148869,0.124094,...,0.297259,0.303152,1.656565,-1.646551,1.262836,0.930126,0.940371,-0.071487,-0.074410,0.961796
1,-0.156061,0.591306,-1.397949,0.987841,-1.231686,-0.887390,-1.633301,0.735855,-1.385308,0.255703,...,-0.836356,-1.432762,1.241678,-0.941083,-1.575093,1.693784,-1.159743,-1.362521,-0.700582,0.622595
2,-0.624560,-0.233004,-1.132794,-0.139077,0.663901,0.659143,-0.134707,0.366280,0.482799,-0.228204,...,-1.627788,1.458085,-1.719528,-1.177299,-0.210058,0.519463,-1.198767,0.897829,-0.077448,-1.426452
3,-0.231424,0.176015,-1.088738,0.850180,-0.242132,0.420693,0.995372,-0.928780,0.418453,-0.577191,...,1.642941,-0.423482,1.231547,1.018242,0.425738,-1.275904,1.498841,1.043248,-0.553736,-0.255119
4,-0.662773,0.185382,-1.310413,1.238650,1.681172,-0.041958,-0.094359,-1.551061,1.283978,-0.008615,...,0.979224,0.891956,1.720278,-0.685381,0.929598,-1.597108,-0.019599,-0.127696,-1.513206,-0.082542


In [12]:
# add custom functions

def _sigmoid(x1):
    """Special case of logistic function to transform to probabilities."""
    with np.errstate(over='ignore', under='ignore'):
        return 1 / (1 + np.exp(-x1))
sig = make_function(function=_sigmoid,
                        name='sig',
                        arity=1)

est_gp = SymbolicRegressor(population_size=30,
                           generations=30, stopping_criteria=0.00,
                           init_depth=(2, 5), init_method='half and half',
                           function_set=('add', 'sub', 'mul', 'div', 'sqrt',
                                         'log', 'abs', 'neg', 'inv', 'max', 
                                         'min', 'sin', 'cos', 'tan', sig),
                           metric='mean absolute error',
                           p_crossover=0.8, p_subtree_mutation=0.05,
                           p_hoist_mutation=0.05, p_point_mutation=0.05,
                           p_point_replace=0.05,
                           verbose=1,
                           parsimony_coefficient=0.01, random_state=0)

start_time = time()
est_gp.fit(X, y)
print("Time Usage:", time()-start_time)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     7.00          3.55811        9         0.848109              N/A      0.77s
   1     4.80          1.10121        8         0.815287              N/A      0.55s
   2     4.43          0.92907        8         0.815287              N/A      0.53s
   3     4.47         0.896344        7         0.812821              N/A      0.53s
   4     4.03         0.894516        6         0.812821              N/A      0.48s
   5     3.93         0.843041        6         0.812331              N/A      0.47s
   6     4.00         0.877267        6         0.812331              N/A      0.44s
   7     4.30         0.986982        6         0.812331              N/A      0.45s
   8     3.83         0.887437        6         0.812331              N/A  